In [1]:
import cv2
import numpy as np
import torch
import torch.nn.functional as F

from tennis_tracker.player_location.tracknet import BallTrackerNet
from tennis_tracker.player_location.postprocess import postprocess
model = BallTrackerNet(out_channels=15)
model.load_state_dict(torch.load("model_tennis_court_det.pt", map_location="mps"))
model.eval()

im = cv2.imread("test2.png")
im = cv2.resize(im, (640, 360))
im = im.astype(np.float32) / 255.
im = np.rollaxis(im, 2, 0)
im = torch.tensor(im)
im = im.unsqueeze(0)

out = model(im)
pred = F.sigmoid(out).cpu().detach().numpy()[0]
points = []
for kps_num in range(14):
    heatmap = (pred[kps_num] * 255).astype(np.uint8)
    x_pred, y_pred = postprocess(heatmap, scale=2, low_thresh=170, max_radius=25)
    points.append((x_pred, y_pred))
print(points)

/var/folders/s3/bqcdtp2s6652tsxqm4hhv4gr0000gn/T/ipykernel_10771/2353270434.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_tenni

[(None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None)]
